In [1]:
import pyreadr
import numpy as onp
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import jax.numpy as jnp
from scipy.stats import linregress
import time

import sys
import os
# sys.path.append("../../../models") # lab server
sys.path.append("../../../contrastive-regression/models/") # lab server
# from linear_cr import LinearCR
from linear_cr_new import LinearContrastiveRegression

In [2]:
def return_indices_of_a(a, b):
  b_set = set(b)
  return [i for i, v in enumerate(a) if v in b_set]

In [3]:
ct = 'L2or3'
df = pyreadr.read_r('../pb/'+ ct + '.rds')[None]
hvg = pyreadr.read_r('../hvg1k/'+ ct +'.rds')[None]
input1 = df.index.values.tolist()
input2 = hvg[None].to_list()
match_id = return_indices_of_a(input1,input2)
# print(len(match_id))
df = df.iloc[match_id]
print(df.shape)

(1000, 41)


In [4]:
score_type = 'scoreA'
output_dir = './' + score_type + '/'
if not os.path.exists(output_dir):
   os.makedirs(output_dir)

score = pyreadr.read_r('../'+ score_type + '_pfc.rds')[None]
print(score.shape)

(20, 5)


In [5]:
X = df[score['sample'][score['diagnosis']=='ASD'].to_list()].to_numpy().T
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(10, 1000)


In [6]:
# A = df[score['sample'][score['diagnosis']=='ASD'].to_list()]
# print(list(A) == score['sample'][score['diagnosis']=='ASD'].to_list())
Y = df[score['sample'][score['diagnosis']=='Control'].to_list()].to_numpy().T
print(type(Y))
print(Y.shape)

<class 'numpy.ndarray'>
(10, 1000)


In [7]:
R = score['zscore'][score['diagnosis']=='ASD']
R = onp.atleast_2d(R).T
#print(R)

In [8]:
# Instantiate model
model = LinearContrastiveRegression()

# Fit model
d = 8 # dimension reduction
start = time.time()
model.fit(X, Y, R, d, seed = 10)
end = time.time()

# Make predictions on training data
preds = model.predict(X)

Step: 0               Log marginal lik.: -2216.19
Step: 200             Log marginal lik.: 20488.7
Step: 400             Log marginal lik.: 40209.67
Step: 600             Log marginal lik.: 58594.05
Step: 800             Log marginal lik.: 70765.83
Step: 1000            Log marginal lik.: 73100.78
Step: 1200            Log marginal lik.: 73167.41
Step: 1400            Log marginal lik.: 73214.52
Step: 1600            Log marginal lik.: 73154.06
Step: 1800            Log marginal lik.: 73252.72
Step: 2000            Log marginal lik.: 73178.45
Step: 2200            Log marginal lik.: 73208.03
Step: 2400            Log marginal lik.: 73267.62
Step: 2600            Log marginal lik.: 73158.12
Step: 2800            Log marginal lik.: 73082.06
Step: 3000            Log marginal lik.: 73194.69
Step: 3200            Log marginal lik.: 73139.65
Step: 3400            Log marginal lik.: 73145.08
Step: 3600            Log marginal lik.: 73167.28


In [9]:
# runtime
runtime = end - start
runtime

842.5425131320953

In [10]:
_, _, r2_contrastivereg, _, _ = linregress(R.squeeze(), preds)
print(r2_contrastivereg)

0.8932320691934508
